# OLGA

OLGA is used to simulate repertoires and confounders. 

OLGA repository: https://github.com/statbiophys/OLGA/tree/master/olga

Running OLGA from the command line with the default model to simulate TCRbeta:

```
olga-generate_sequences --humanTRB -n 1000 -o rep_1.tsv --seed=1
```

Running OLGA from the command line with the custom model:

```
olga-generate_sequences -n 1000 -o rep_1.tsv --seed=1 --set_custom_model_VDJ ./path_to_model_folder/
```

The custom model is located under ./olga_model_removed_TRBV5_1/.

# immuneML 

immuneML is used to load the repertoire and implant signals in the repertoires, to simulate the immune state.



In [8]:
from pathlib import Path
from util.load_repertoire import load_olga_repertoire

repertoire = load_olga_repertoire(filepath=Path("./data/rep1.tsv"), result_path=Path("./data/immuneML_format/"))

print(f"Repertoire id: {repertoire.identifier}\nMetadata: {repertoire.metadata}")

Repertoire id: be2c98ff1dcf47df98d35c43804e8ef2
Metadata: {'field_list': ['sequence_aas', 'sequences', 'v_genes', 'j_genes', 'v_subgroups', 'j_subgroups', 'v_alleles', 'j_alleles', 'chains', 'counts', 'sequence_identifiers'], 'filename': 'rep1.npy'}


## A new repertoire with a signal

In [9]:
from util.implanting import make_default_signal, make_repertoire_with_signal
from immuneML.util.PathBuilder import PathBuilder

signal = make_default_signal()

result_path = PathBuilder.build("./data/implanted/")

new_repertoire = make_repertoire_with_signal(repertoire, signal, result_path)

print(f"Repertoire id: {new_repertoire.identifier}\nMetadata: {new_repertoire.metadata}")

Repertoire id: 17bdbb68aabf4e899ffab91a1a3749d6
Metadata: {'field_list': ['immune_state', 'sequence_aas', 'sequences', 'v_genes', 'j_genes', 'v_subgroups', 'j_subgroups', 'v_alleles', 'j_alleles', 'chains', 'counts', 'frame_types', 'sequence_identifiers'], 'filename': '17bdbb68aabf4e899ffab91a1a3749d6.npy', 'immune_state': True}


## A new repertoire with no signal

In [10]:
from util.implanting import make_repertoire_without_signal

new_repertoire_no_signal = make_repertoire_without_signal(repertoire=repertoire, signal_name=signal.id, path=result_path)

print(f"Repertoire id: {new_repertoire_no_signal.identifier}\nMetadata: {new_repertoire_no_signal.metadata}")

Repertoire id: fa51bbefe88b4e6a90dbec63305be0b6
Metadata: {'field_list': ['sequence_aas', 'sequences', 'v_genes', 'j_genes', 'v_subgroups', 'j_subgroups', 'v_alleles', 'j_alleles', 'chains', 'counts', 'frame_types', 'sequence_identifiers'], 'filename': 'fa51bbefe88b4e6a90dbec63305be0b6.npy', 'immune_state': False}


## Making a dataset

In [11]:
from util.implanting import make_AIRR_dataset

dataset = make_AIRR_dataset(repertoires=[new_repertoire, new_repertoire_no_signal], path=Path("./data/exporting"), 
                           dataset_name="new_dataset", signal_name=signal.id)

print(dataset.labels)

{'immune_state': [True, False]}


Exported 

In [6]:
import os
import dagsim.baseDS as ds
import numpy as np
import uuid
from pathlib import Path
from util.load_repertoire import load_olga_repertoire, load_iml_repertoire
from util.implanting import make_default_signal, make_repertoire_with_signal
from immuneML.util.PathBuilder import PathBuilder
from util.implanting import make_repertoire_without_signal
from util.implanting import make_AIRR_dataset

# os.chdir("./simData")
result_path = PathBuilder.build("./data/implanted/")
signal = make_default_signal()

In [7]:
def get_state(confounder):
    return np.random.binomial(n=1, p=0.5) if confounder==1 else np.random.binomial(n=1, p=0.2)

def generate_repertoire(state, confounder):
    uid = str(uuid.uuid4())
    seed = np.random.randint(0,10e5)
    # path = "healthy" if state==1 else "disease"
    if confounder==1:
        os.system("olga-generate_sequences --humanTRB -n 1000 -o ./data/" + uid + ".tsv --seed=" + str(seed))
    else:
        os.system("olga-generate_sequences -n 1000 -o ./data/" + uid + ".tsv --seed=" + str(seed) + " --set_custom_model_VDJ ./olga_model_removed_TRBV5_1")

    rep = load_olga_repertoire(filepath=Path("./data/" + uid + ".tsv"), result_path=Path("./data/immuneML_format/"))
    
    if state==0:
        make_repertoire_without_signal(repertoire=rep, signal_name=signal.id, path=result_path)
    else:
        make_repertoire_with_signal(rep, signal, result_path)

    return uid

In [8]:
Confounder = ds.Generic(name="sex", function=np.random.binomial, arguments={"n": 1, "p":0.5})
ImmuneState = ds.Generic(name="state", function=get_state, arguments={"confounder": Confounder})
Repertoire = ds.Generic(name="repertoire", function=generate_repertoire, arguments={"state": ImmuneState, "confounder": Confounder})

In [9]:
CAIRR_graph = ds.Graph(name="CAIRR_graph", list_nodes=[Confounder, ImmuneState, Repertoire])
CAIRR_data = CAIRR_graph.simulate(num_samples=10, csv_name="CAIRR_data")

Starting sequence generation... 
Completed generating all 1000 sequences in 0.07 seconds.
Starting sequence generation... 
Completed generating all 1000 sequences in 0.06 seconds.
Starting sequence generation... 
Completed generating all 1000 sequences in 0.06 seconds.
Starting sequence generation... 
Completed generating all 1000 sequences in 0.06 seconds.
Starting sequence generation... 
Completed generating all 1000 sequences in 0.07 seconds.
Starting sequence generation... 
Completed generating all 1000 sequences in 0.06 seconds.
Starting sequence generation... 
Completed generating all 1000 sequences in 0.06 seconds.
Starting sequence generation... 
Completed generating all 1000 sequences in 0.06 seconds.
Starting sequence generation... 
Completed generating all 1000 sequences in 0.06 seconds.
Starting sequence generation... 
Completed generating all 1000 sequences in 0.06 seconds.


In [11]:
dataset = make_AIRR_dataset(repertoires=[load_iml_repertoire(filepath=Path("./data/immuneML_format/"), identifier=id) for id in CAIRR_data["repertoire"]], path=Path("./data/exporting"),
                           dataset_name="new_dataset", signal_name=signal.id)

AssertionError: Repertoire: the file representing the repertoire has to be in numpy binary format. Got  instead.

AssertionError: Repertoire: the file representing the repertoire has to be in numpy binary format. Got  instead.